In [8]:
from keras.models import Model
from keras import backend as K
from keras import layers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Conv2DTranspose
from keras.models import Model
import numpy as np

Using TensorFlow backend.


## CAE Conv2DTranspose

In [106]:
def CAE_Conv2DTranspose(input_shape=(128, 128, 1), filters=[8, 16, 32, 64, 128, 256, 512, 1024, 30]):

    input_img = Input(shape=input_shape)

    # Encoder
    x = Conv2D(filters[0], 5, strides=2, padding='same', activation='relu',name='conv1', input_shape=input_shape)(input_img)
    x = Conv2D(filters[1], 3, strides=2, padding='same',activation='relu', name='conv2')(x)
    x = Conv2D(filters[2], 3, strides=2, padding='same',activation='relu', name='conv3')(x)
    x = Conv2D(filters[3], 3, strides=2, padding='same', activation='relu', name='conv4')(x)
    # x = Conv2D(filters[4], 3, strides=2, padding='same', activation='relu', name='conv5')(x)

    x = Flatten()(x)

    encoded = Dense(units=filters[-1], name='embedding')(x)

    y = Dense(units=3, name='input_clustering')(encoded)

    # Decoder
    x = Dense(units= 4096, #filters[5]*int(input_shape[0]/64) * int(input_shape[0]/64)
            activation='relu')(encoded)

    # Reshape into an image of the same shape as before our last `Flatten` layer
    x = Reshape((8, 8, 64))(x)

    # x = Conv2DTranspose(filters[4], 3, strides=2, padding='same', activation='relu', name='deconv5')(x)
    x = Conv2DTranspose(filters[2], 3, strides=2, padding='same', activation='relu', name='deconv4')(x)
    x = Conv2DTranspose(filters[1], 3, strides=2, padding='same', activation='relu', name='deconv3')(x)
    x = Conv2DTranspose(filters[0], 5, strides=2, padding='same', activation='relu', name='deconv2')(x)

    decoded = Conv2DTranspose(1, 3, strides=2, padding='same', name='deconv1')(x)

    return Model(inputs=input_img, outputs=decoded, name='CAE_Conv2DTranspose'), Model(inputs=input_img, outputs=[encoded, y], name='CE')

In [107]:
autoencoder, encoder = CAE_Conv2DTranspose()
encoder.summary()
autoencoder.summary()

Model: "CE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_48 (InputLayer)        (None, 128, 128, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 8)         208       
_________________________________________________________________
conv2 (Conv2D)               (None, 32, 32, 16)        1168      
_________________________________________________________________
conv3 (Conv2D)               (None, 16, 16, 32)        4640      
_________________________________________________________________
conv4 (Conv2D)               (None, 8, 8, 64)          18496     
_________________________________________________________________
flatten_48 (Flatten)         (None, 4096)              0         
_________________________________________________________________
embedding (Dense)            (None, 30)                122910   

In [6]:
def CAE_Conv2DTranspose_cropped(input_shape=(64, 64, 1), filters=[32, 64, 128, 256, 512, 1024, 30]):
    
    '''
    CAE_Upsampling2D convolutional autoencoder model.
    
    Arguments:
        img_shape: (128, 128, 1)
        filters: [32, 64, 128, 256, 512, 1024, 3]
    
    return:
        Models of autoencoder and encoder
            (autoencoder, encoder)
            
        outputs:
            autoencoder: encoded
            encoder: decoded
    '''
    
    input_img = Input(shape=input_shape)
    
    #Encoder
    x = Conv2D(filters[0], 5, strides=3, padding='same', activation='relu', name='conv1', input_shape=input_shape)(input_img)
    x = Conv2D(filters[1], 3, strides=2, padding='same', activation='relu', name='conv2')(x)
    x = Conv2D(filters[2], 3, strides=2, padding='same', activation='relu', name='conv3')(x)
    x = Conv2D(filters[3], 3, strides=2, padding='same', activation='relu', name='conv4')(x)
    x = Conv2D(filters[4], 3, strides=2, padding='same', activation='relu', name='conv5')(x)
    x = Conv2D(filters[5], 3, strides=2, padding='same', activation='relu', name='conv6')(x)
    
    x = Flatten()(x)
    
    encoded = Dense(units=filters[-1], name='embedding')(x)
    
    #Decoder
    x = Dense(units=filters[5]*int(input_shape[0]/64)*int(input_shape[0]/64), activation='relu')(encoded)

    # Reshape into an image of the same shape as before our last `Flatten` layer
    x = Reshape((int(input_shape[0]/64), int(input_shape[0]/64), filters[5]))(x)
    
    x = Conv2DTranspose(filters[4], 3, strides=2, padding='same', activation='relu', name='deconv6')(x)
    x = Conv2DTranspose(filters[3], 3, strides=2, padding='same', activation='relu', name='deconv5')(x)
    x = Conv2DTranspose(filters[2], 3, strides=2, padding='same', activation='relu', name='deconv4')(x)
    x = Conv2DTranspose(filters[1], 3, strides=2, padding='same', activation='relu', name='deconv3')(x)
    x = Conv2DTranspose(filters[0], 3, strides=2, padding='same', activation='relu', name='deconv2')(x)

    decoded = Conv2DTranspose(input_shape[2], 3, strides=2, padding='same', name='deconv1')(x)
    
    return Model(inputs=input_img, outputs=decoded, name='CAE_Conv2DTranspose'), Model(inputs=input_img, outputs=encoded, name='CE')

In [7]:
autoencoder, encoder = CAE_Conv2DTranspose_cropped()
encoder.summary()
autoencoder.summary()

NameError: name 'Input' is not defined

In [5]:
def CAE_Conv2DTranspose(input_shape=(128, 128, 1), filters=[8, 16, 32, 64, 128, 256, 512, 1024, 3]):
    
    '''
    CAE_Upsampling2D convolutional autoencoder model.
    
    Arguments:
        img_shape: (128, 128, 1)
        filters: [32, 64, 128, 256, 512, 1024, 3]
    
    return:
        Models of autoencoder and encoder
            (autoencoder, encoder)
            
        outputs:
            autoencoder: encoded
            encoder: decoded
    '''
    
    input_img = Input(shape=input_shape)
    
    #Encoder
    x = Conv2D(filters[0], 5, strides=2, padding='same', activation='relu', name='conv1', input_shape=input_shape)(input_img)
    x = Conv2D(filters[1], 3, strides=2, padding='same', activation='relu', name='conv2')(x)
    x = Conv2D(filters[2], 3, strides=2, padding='same', activation='relu', name='conv3')(x)
    x = Conv2D(filters[3], 3, strides=2, padding='same', activation='relu', name='conv4')(x)
    x = Conv2D(filters[4], 3, strides=2, padding='same', activation='relu', name='conv5')(x)
    x = Conv2D(filters[5], 3, strides=2, padding='same', activation='relu', name='conv6')(x)
    
    x = Flatten()(x)
    
    encoded = Dense(units=filters[-1], name='embedding')(x)
    
    #Decoder
    x = Dense(units=filters[5]*int(input_shape[0]/64)*int(input_shape[0]/64), activation='relu')(encoded)

    # Reshape into an image of the same shape as before our last `Flatten` layer
    x = Reshape((int(input_shape[0]/64), int(input_shape[0]/64), filters[5]))(x)
    
    x = Conv2DTranspose(filters[4], 3, strides=2, padding='same', activation='relu', name='deconv6')(x)
    x = Conv2DTranspose(filters[3], 3, strides=2, padding='same', activation='relu', name='deconv5')(x)
    x = Conv2DTranspose(filters[2], 5, strides=2, padding='same', activation='relu', name='deconv4')(x)
    x = Conv2DTranspose(filters[1], 5, strides=2, padding='same', activation='relu', name='deconv3')(x)
    x = Conv2DTranspose(filters[0], 5, strides=2, padding='same', activation='relu', name='deconv2')(x)

    decoded = Conv2DTranspose(input_shape[2], 3, strides=2, padding='same', name='deconv1')(x)
    
    return Model(inputs=input_img, outputs=decoded, name='CAE_Conv2DTranspose'), Model(inputs=input_img, outputs=encoded, name='CE')

In [158]:
autoencoder, encoder = CAE_Conv2DTranspose()
encoder.summary()
autoencoder.summary()

Model: "CE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_74 (InputLayer)        (None, 128, 128, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 8)         208       
_________________________________________________________________
conv2 (Conv2D)               (None, 32, 32, 16)        1168      
_________________________________________________________________
conv3 (Conv2D)               (None, 16, 16, 32)        4640      
_________________________________________________________________
conv4 (Conv2D)               (None, 8, 8, 64)          18496     
_________________________________________________________________
conv5 (Conv2D)               (None, 4, 4, 128)         73856     
_________________________________________________________________
conv6 (Conv2D)               (None, 2, 2, 256)         295168   

In [151]:
def CAE_Conv2DTranspose_small(input_shape=(128, 128, 1), filters=[4, 8, 16, 32, 64, 128, 256, 512, 1024, 3]):
    
    '''
    CAE_Upsampling2D convolutional autoencoder model.
    
    Arguments:
        img_shape: (128, 128, 1)
        filters: [32, 64, 128, 256, 512, 1024, 3]
    
    return:
        Models of autoencoder and encoder
            (autoencoder, encoder)
            
        outputs:
            autoencoder: encoded
            encoder: decoded
    '''
    
    input_img = Input(shape=input_shape)
    
    #Encoder
    x = Conv2D(filters[0], 5, strides=3, padding='same', activation='relu', name='conv1', input_shape=input_shape)(input_img)
    x = Conv2D(filters[1], 5, strides=2, padding='same', activation='relu', name='conv2')(x)
    x = Conv2D(filters[2], 3, strides=2, padding='same', activation='relu', name='conv3')(x)
    x = Conv2D(filters[3], 3, strides=2, padding='same', activation='relu', name='conv4')(x)
    
    x = Flatten()(x)
    
    encoded = Dense(units=filters[-1], name='embedding')(x)
    
    #Decoder
    x = Dense(units=filters[7], activation='relu')(encoded)

    # Reshape into an image of the same shape as before our last `Flatten` layer
    x = Reshape((4, 4, filters[3]))(x)
    
    x = Conv2DTranspose(filters[4], 3, strides=2, padding='same', activation='relu', name='deconv5')(x)
    x = Conv2DTranspose(filters[3], 3, strides=2, padding='same', activation='relu', name='deconv4')(x)
    x = Conv2DTranspose(filters[2], 3, strides=2, padding='same', activation='relu', name='deconv3')(x)
    x = Conv2DTranspose(filters[1], 3, strides=2, padding='same', activation='relu', name='deconv2')(x)

    decoded = Conv2DTranspose(input_shape[2], 3, strides=2, padding='same', name='deconv1')(x)
    
    return Model(inputs=input_img, outputs=decoded, name='CAE_Conv2DTranspose'), Model(inputs=input_img, outputs=encoded, name='CE')

In [152]:
autoencoder, encoder = CAE_Conv2DTranspose_small()
encoder.summary()
autoencoder.summary()

Model: "CE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_71 (InputLayer)        (None, 128, 128, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 43, 43, 4)         104       
_________________________________________________________________
conv2 (Conv2D)               (None, 22, 22, 8)         808       
_________________________________________________________________
conv3 (Conv2D)               (None, 11, 11, 16)        1168      
_________________________________________________________________
conv4 (Conv2D)               (None, 6, 6, 32)          4640      
_________________________________________________________________
flatten_72 (Flatten)         (None, 1152)              0         
_________________________________________________________________
embedding (Dense)            (None, 3)                 3459     

## CAE UpSampling2D

In [76]:
def CAE_Upsampling2D(input_shape=(128, 128, 1), filters=[32, 64, 128, 256, 512, 1024, 3]):
    
    '''
    CAE_Upsampling2D convolutional autoencoder model.
    Arguments:
        img_shape: (128, 128, 1)
        filters: [32, 64, 128, 256, 512, 1024, 3]
    return:
        (autoencoder, encoder): Models of autoencoder and encoder
    '''
    
    input_img = Input(shape=input_shape)
    #Encoder
    x = Conv2D(filters[0], 3, strides=2, padding='same', activation='relu', name='conv1', input_shape=input_shape)(input_img)
    x = Conv2D(filters[1], 3, strides=2, padding='same', activation='relu', name='conv2')(x)
    x = Conv2D(filters[2], 3, strides=2, padding='same', activation='relu', name='conv3')(x)
    x = Conv2D(filters[3], 3, strides=2, padding='same', activation='relu', name='conv4')(x)
    x = Conv2D(filters[4], 3, strides=2, padding='same', activation='relu', name='conv5')(x)
    x = Conv2D(filters[5], 3, strides=2, padding='same', activation='relu', name='conv6')(x)
    shape_before_flattening = K.int_shape(x)
    
    x = Flatten()(x)
    
    encoded = Dense(units=filters[6], name='embedding')(x)
    
    #Decoder
    x = Dense(units=filters[5]*int(input_shape[0]/32)*int(input_shape[0]/32), activation='relu')(encoded)

    # Reshape into an image of the same shape as before our last `Flatten` layer
    x = Reshape((int(input_shape[0]/32), int(input_shape[0]/32), filters[5]))(x)

    x = Conv2D(filters[5], 3, padding='same', activation='relu', name='conv7')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters[4], 3, padding='same', activation='relu', name='conv8')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters[3], 3, padding='same', activation='relu', name='conv9')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters[2], 3, padding='same', activation='relu', name='conv10')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters[1], 3, padding='same', activation='relu', name='conv11')(x)
    x = UpSampling2D((2, 2))(x)
    
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    return Model(inputs=input_img, outputs=decoded, name='CAE_Upsampling2D'), Model(inputs=input_img, outputs=encoded, name='CE')


In [78]:
autoencoder, encoder = CAE_Upsampling2D()
encoder.summary()
autoencoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_39 (InputLayer)        (None, 128, 128, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        320       
_________________________________________________________________
conv2 (Conv2D)               (None, 32, 32, 64)        18496     
_________________________________________________________________
conv3 (Conv2D)               (None, 16, 16, 128)       73856     
_________________________________________________________________
conv4 (Conv2D)               (None, 8, 8, 256)         295168    
_________________________________________________________________
conv5 (Conv2D)               (None, 4, 4, 512)         1180160   
_________________________________________________________________
conv6 (Conv2D)               (None, 2, 2, 1024)        4719

## CAE Conv2DTranspose (old)

In [9]:
def autoencoderConv2D_1(input_shape=(128, 128, 1), filters=[32, 64, 128, 256, 3]):
    input_img = Input(shape=input_shape)
    if input_shape[0] % 8 == 0:
        pad3 = 'same'
    else:
        pad3 = 'valid'
    x = Conv2D(filters[0], 5, strides=2, padding='same', activation='relu', name='conv1', input_shape=input_shape)(input_img)

    x = Conv2D(filters[1], 5, strides=2, padding='same', activation='relu', name='conv2')(x)

    x = Conv2D(filters[2], 3, strides=2, padding='same', activation='relu', name='conv3')(x)
    
    x = Flatten()(x)
    
    encoded = Dense(units=filters[3], name='embedding')(x)
    
    y = Dense(units=filters[4], name='output_to_cluster')(encoded)
    
    x = Dense(units=filters[2]*int(input_shape[0]/8)*int(input_shape[0]/8), activation='relu')(encoded)

    x = Reshape((int(input_shape[0]/8), int(input_shape[0]/8), filters[2]))(x)
    
    x = Conv2DTranspose(filters[1], 3, strides=2, padding='same', activation='relu', name='deconv3')(x)

    x = Conv2DTranspose(filters[0], 5, strides=2, padding='same', activation='relu', name='deconv2')(x)

    decoded = Conv2DTranspose(input_shape[2], 5, strides=2, padding='same', name='deconv1')(x)
    
    return Model(inputs=input_img, outputs=decoded, name='AE'), Model(inputs=input_img, outputs=y, name='encoder')

In [10]:
autoencoder, encoder = autoencoderConv2D_1()
encoder.summary()
autoencoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        832       
_________________________________________________________________
conv2 (Conv2D)               (None, 32, 32, 64)        51264     
_________________________________________________________________
conv3 (Conv2D)               (None, 16, 16, 128)       73856     
_________________________________________________________________
flatten_3 (Flatten)          (None, 32768)             0         
_________________________________________________________________
embedding (Dense)            (None, 256)               8388864   
_________________________________________________________________
output_to_cluster (Dense)    (None, 3)                 771 

In [13]:
from keras.models import Sequential, Model

def CAE(input_shape=(28, 28, 1), filters=[32, 64, 128, 10]):
    model = Sequential()
    if input_shape[0] % 8 == 0:
        pad3 = 'same'
    else:
        pad3 = 'valid'
    model.add(Conv2D(filters[0], 5, strides=2, padding='same', activation='relu', name='conv1', input_shape=input_shape))

    model.add(Conv2D(filters[1], 5, strides=2, padding='same', activation='relu', name='conv2'))

    model.add(Conv2D(filters[2], 3, strides=2, padding=pad3, activation='relu', name='conv3'))

    model.add(Flatten())
    model.add(Dense(units=filters[3], name='embedding'))
    model.add(Dense(units=filters[2]*int(input_shape[0]/8)*int(input_shape[0]/8), activation='relu'))

    model.add(Reshape((int(input_shape[0]/8), int(input_shape[0]/8), filters[2])))
    model.add(Conv2DTranspose(filters[1], 3, strides=2, padding=pad3, activation='relu', name='deconv3'))

    model.add(Conv2DTranspose(filters[0], 5, strides=2, padding='same', activation='relu', name='deconv2'))

    model.add(Conv2DTranspose(input_shape[2], 5, strides=2, padding='same', name='deconv1'))
    model.summary()
    return model

In [14]:
autoencoder, encoder = CAE()
encoder.summary()
autoencoder.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 14, 14, 32)        832       
_________________________________________________________________
conv2 (Conv2D)               (None, 7, 7, 64)          51264     
_________________________________________________________________
conv3 (Conv2D)               (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten_4 (Flatten)          (None, 1152)              0         
_________________________________________________________________
embedding (Dense)            (None, 10)                11530     
_________________________________________________________________
dense_4 (Dense)              (None, 1152)              12672     
_________________________________________________________________
reshape_4 (Reshape)          (None, 3, 3, 128)        

TypeError: cannot unpack non-iterable Sequential object